<a href="https://www.kaggle.com/code/ravaghi/cibmtr-autogluon-classifier-training?scriptVersionId=226032266" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports and configs

In [1]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import KFold
import pandas as pd
import warnings
import joblib
import shutil

warnings.filterwarnings("ignore")

In [2]:
class CFG:
    train_path = "/kaggle/input/equity-post-HCT-survival-predictions/train.csv"
    test_path = "/kaggle/input/equity-post-HCT-survival-predictions/test.csv"
    sample_sub_path = "/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv"

    n_folds = 10
    seed = 42

    time_limit = 3600 * 11
    run_name = "ag_classifier"

# Loading and preprocessing data

In [3]:
train = pd.read_csv(CFG.train_path)
test = pd.read_csv(CFG.test_path)

In [4]:
kf = KFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
split = kf.split(train, train.efs)

for i, (train_index, val_index) in enumerate(split):
    train.loc[val_index, "fold"] = i

In [5]:
train = train.drop(["ID", "efs_time"], axis=1)
test = test.drop(["ID"], axis=1)

# Training

In [6]:
predictor = TabularPredictor(
    path=f"/{CFG.run_name}",
    problem_type="binary",
    eval_metric="roc_auc",
    label="efs",
    groups="fold",
    verbosity=2
)

In [7]:
predictor.fit(
    train_data=train,
    time_limit=CFG.time_limit,
    presets="best_quality",
    excluded_model_types=["KNN"],
    keep_only_best=True
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.27 GB / 31.35 GB (96.5%)
Disk Space Avail:   1892.08 GB / 8062.39 GB (23.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stac

(_ray_fit pid=5557) [1000]	valid_set's binary_logloss: 0.576285
(_ray_fit pid=5688) [1000]	valid_set's binary_logloss: 0.582957
(_ray_fit pid=5684) [1000]	valid_set's binary_logloss: 0.574749
(_ray_fit pid=5816) [1000]	valid_set's binary_logloss: 0.594954
(_ray_fit pid=5848) [1000]	valid_set's binary_logloss: 0.593652


(_dystack pid=170) 	0.7515	 = Validation score   (roc_auc)
(_dystack pid=170) 	33.21s	 = Training   runtime
(_dystack pid=170) 	3.66s	 = Validation runtime
(_dystack pid=170) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 4971.21s of the 8271.63s of remaining time.
(_dystack pid=170) 	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.20%)
(_ray_fit pid=5894) No improvement since epoch 8: early stopping
(_ray_fit pid=5892) No improvement since epoch 12: early stopping
(_ray_fit pid=5891) No improvement since epoch 13: early stopping
(_ray_fit pid=5893) No improvement since epoch 17: early stopping
(_ray_fit pid=6040) No improvement since epoch 14: early stopping
(_ray_fit pid=6109) No improvement since epoch 12: early stopping
(_ray_fit pid=6078) No improvement since epoch 15: early stopping
(_ray_fit pid=6154) No improvement since epoch 15: early stopping
(_ray_fit pid=6229) No impro

(_ray_fit pid=6703) [1000]	valid_set's binary_logloss: 0.592918
(_ray_fit pid=6700) [1000]	valid_set's binary_logloss: 0.579456
(_ray_fit pid=6702) [2000]	valid_set's binary_logloss: 0.572763 [repeated 3x across cluster]
(_ray_fit pid=6829) [1000]	valid_set's binary_logloss: 0.577684
(_ray_fit pid=6859) [1000]	valid_set's binary_logloss: 0.569716
(_ray_fit pid=6829) [2000]	valid_set's binary_logloss: 0.575422 [repeated 2x across cluster]
(_ray_fit pid=6926) [2000]	valid_set's binary_logloss: 0.591487 [repeated 4x across cluster]
(_ray_fit pid=6960) [2000]	valid_set's binary_logloss: 0.59193 [repeated 3x across cluster]


(_dystack pid=170) 	0.7557	 = Validation score   (roc_auc)
(_dystack pid=170) 	56.84s	 = Training   runtime
(_dystack pid=170) 	4.44s	 = Validation runtime
(_dystack pid=170) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 4359.24s of the 7659.66s of remaining time.
(_dystack pid=170) 	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.11%)
(_dystack pid=170) 	0.7434	 = Validation score   (roc_auc)
(_dystack pid=170) 	283.28s	 = Training   runtime
(_dystack pid=170) 	0.96s	 = Validation runtime
(_dystack pid=170) Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 4073.48s of the 7373.90s of remaining time.
(_dystack pid=170) 	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.72%)
(_dystack pid=170) 	0.7459	 = Validation score   (roc_auc)
(_dystack pid=170) 	67.29s	 = Training   runtime
(_dystack pi

In [8]:
predictor.leaderboard(silent=True).style.background_gradient(subset=["score_val"], cmap="RdYlGn")

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.762912,roc_auc,142.978693,12778.379022,0.005069,4.515876,3,True,57
1,CatBoost_r69_BAG_L2,0.761883,roc_auc,132.784349,10345.606252,0.288014,161.757357,2,True,53
2,CatBoost_r137_BAG_L1,0.761100,roc_auc,0.491930,809.562954,0.491930,809.562954,1,True,14
3,CatBoost_r69_BAG_L1,0.760874,roc_auc,0.469331,758.789655,0.469331,758.789655,1,True,18
4,CatBoost_r60_BAG_L1,0.760755,roc_auc,0.426489,689.239847,0.426489,689.239847,1,True,42
5,CatBoost_r5_BAG_L1,0.760633,roc_auc,0.251590,126.639145,0.251590,126.639145,1,True,36
6,XGBoost_r89_BAG_L2,0.760544,roc_auc,133.211981,10212.708956,0.715646,28.860061,2,True,52
7,NeuralNetFastAI_r95_BAG_L2,0.760380,roc_auc,134.579776,10599.510931,2.083441,415.662036,2,True,56
8,CatBoost_r49_BAG_L1,0.759944,roc_auc,0.245773,113.609680,0.245773,113.609680,1,True,25
9,NeuralNetFastAI_BAG_L2,0.759920,roc_auc,133.953633,10340.442520,1.457298,156.593626,2,True,50


# Saving predictor and OOF preds

In [9]:
oof_preds = predictor.predict_oof()
oof_pred_probs = predictor.predict_proba_oof()

In [10]:
joblib.dump(oof_preds, "oof_preds.pkl")
joblib.dump(oof_pred_probs, "oof_pred_probs.pkl")

['oof_pred_probs.pkl']

In [11]:
shutil.make_archive(
    f"/kaggle/working/{CFG.run_name}", 
    "zip", 
    f"/{CFG.run_name}"
)

'/kaggle/working/ag_classifier.zip'